<a href="https://colab.research.google.com/github/fpgmina/gibbs/blob/main/gibbs_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.stats import invgamma, gamma
from typing import Tuple

In [3]:
def gibbs_sharpe_student(
    returns: np.ndarray,
    nu: float = 3.0,
    n_iter: int = 5000,
    burn_in: int = 500,
    mu0: float = 0.0,
    tau2: float = 1000.0,
    a0: float = 2.0,
    b0: float = 2.0,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Bayesian Sharpe ratio estimation using Gibbs sampling for t-distributed returns.

    Uses data augmentation: the t-distribution is represented as a scale mixture of normals
    with latent scaling variables lambda_i.

    Model:
        r_i | mu, sigma^2, lambda_i ~ N(mu, sigma^2 / lambda_i)
        lambda_i ~ Gamma(nu/2, nu/2) independently
        mu ~ N(mu0, tau2)
        sigma^2 ~ Inverse-Gamma(a0, b0)
        Sharpe ratio S = mu / sqrt(sigma^2)

    Parameters:
        returns (np.ndarray): 1D array of observed returns.
        nu (float): Degrees of freedom for the t-distribution (typical fat-tailed: 2 < nu < 10).
        n_iter (int): Number of Gibbs iterations.
        burn_in (int): Number of burn-in samples to discard.
        mu0 (float): Prior mean for mu (location parameter).
        tau2 (float): Prior variance for mu.
        a0 (float): Shape parameter for Inverse-Gamma prior on sigma^2.
        b0 (float): Scale parameter for Inverse-Gamma prior on sigma^2.
        seed (int): Random seed for reproducibility.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray]:
            Arrays of posterior samples (after burn-in) for mu, sigma^2, and Sharpe ratio.
    """
    np.random.seed(seed)
    n = len(returns)

    # Initializations
    mu = np.mean(returns)
    sigma2 = np.var(returns, ddof=1)
    lambdas = np.ones(n)  # latent variables

    mu_samples = np.zeros(n_iter)
    sigma2_samples = np.zeros(n_iter)
    sharpe_samples = np.zeros(n_iter)

    for i in range(n_iter):
        # 1. Update latent variables lambda_i | rest
        resids = returns - mu
        # Each lambda_i ~ Gamma((nu+1)/2, (nu + resid_i^2/sigma2)/2)
        shape = (nu + 1) / 2.0
        scale = 2.0 / (nu + (resids ** 2) / sigma2)  # gamma in scipy uses scale=1/rate
        lambdas = gamma.rvs(a=shape, scale=scale)

        # 2. Update mu | rest (Normal)
        tau2_n = 1.0 / (1.0 / tau2 + np.sum(lambdas) / sigma2)
        mu_n = tau2_n * (mu0 / tau2 + np.sum(lambdas * returns) / sigma2)
        mu = np.random.normal(mu_n, np.sqrt(tau2_n))

        # 3. Update sigma^2 | rest (Inverse-Gamma)
        a_n = a0 + n / 2.0
        b_n = b0 + 0.5 * np.sum(lambdas * (returns - mu) ** 2)
        sigma2 = invgamma.rvs(a=a_n, scale=b_n)

        # Store
        mu_samples[i] = mu
        sigma2_samples[i] = sigma2
        sharpe_samples[i] = mu / np.sqrt(sigma2)

    # Discard burn-in
    return (
        mu_samples[burn_in:],
        sigma2_samples[burn_in:],
        sharpe_samples[burn_in:]
    )
